# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import plotly.express as px

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp (K),Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Bredasdorp,-34.5322,20.0403,291.40,96,76,1.21,ZA,1706672733
1,1,College,64.8569,-147.8028,246.14,61,75,1.54,US,1706672734
2,2,Vila Velha,-20.3297,-40.2925,300.12,94,0,3.60,BR,1706672734
3,3,Quellón,-43.1167,-73.6167,287.33,93,96,2.39,CL,1706672605
4,4,Bilibino,68.0546,166.4372,239.43,99,100,0.16,RU,1706672735


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    x='Lng',
    y='Lat',
    geo=True,
    tiles='CartoLight',
    color='City',
    cmap='Category10',
    size='Humidity',
    title='City Map',
    legend='top_right',
    frame_width=800,
    frame_height=500
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
criteria_df = city_data_df[(city_data_df['Humidity'] < 50) & (city_data_df['Max Temp (K)'] < 288.70) & (city_data_df['Wind Speed'] < 2)]

# Drop any rows with null values
filtered_city_data_df = criteria_df.dropna()

# Display sample data
filtered_city_data_df

,City_ID,City,Lat,Lng,Max Temp (K),Humidity,Cloudiness,Wind Speed,Country,Date
127,127,Aral,40.5156,81.2636,272.21,44,100,1.55,CN,1706672793
145,145,Ouarzazate,30.9189,-6.8934,279.31,26,0,1.54,MA,1706672801
307,307,Ixtlahuacán del Río,20.8667,-103.2500,287.07,31,100,0.76,MX,1706672876
340,340,Saint Paul Harbor,57.7900,-152.4072,261.88,48,0,0.00,US,1706672892
373,373,Florence,43.7667,11.2500,280.98,45,98,0.67,IT,1706672906
463,463,Midland,31.9974,-102.0779,287.39,46,0,1.54,US,1706672827
502,502,Loreto,22.2667,-101.9667,288.35,43,100,0.19,MX,1706672971


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
127,Aral,CN,40.5156,81.2636,44,
145,Ouarzazate,MA,30.9189,-6.8934,26,
307,Ixtlahuacán del Río,MX,20.8667,-103.2500,31,
340,Saint Paul Harbor,US,57.7900,-152.4072,48,
373,Florence,IT,43.7667,11.2500,45,
463,Midland,US,31.9974,-102.0779,46,
502,Loreto,MX,22.2667,-101.9667,43,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key, 
    "radius": radius,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Aral - nearest hotel: No hotel found
Ouarzazate - nearest hotel: Fint
Ixtlahuacán del Río - nearest hotel: Hotel Ramirez
Saint Paul Harbor - nearest hotel: Shelikof Lodge
Florence - nearest hotel: Old bridge Apartment
Midland - nearest hotel: DoubleTree by Hilton Hotel Midland Plaza
Loreto - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
127,Aral,CN,40.5156,81.2636,44,No hotel found
145,Ouarzazate,MA,30.9189,-6.8934,26,Fint
307,Ixtlahuacán del Río,MX,20.8667,-103.2500,31,Hotel Ramirez
340,Saint Paul Harbor,US,57.7900,-152.4072,48,Shelikof Lodge
373,Florence,IT,43.7667,11.2500,45,Old bridge Apartment
463,Midland,US,31.9974,-102.0779,46,DoubleTree by Hilton Hotel Midland Plaza
502,Loreto,MX,22.2667,-101.9667,43,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
fig = px.scatter_mapbox(
    hotel_df,
    lat="Lat",
    lon="Lng",
    hover_name="City",
    hover_data={"Hotel Name": True, "Country": True},
    title="Map with dots for hotels",
    mapbox_style="open-street-map",
    zoom=0,
)

# Display the map
fig.show()